# Co-Registration

In [ ]:
from plot_helpers import *
from source_files_extended import sfm_snow_dsm, load_co_reg_dem, load_control_surfaces
from raster_compare.base import RasterFile, MedianAbsoluteDeviation

In [ ]:
control_surfaces_values = load_control_surfaces()

In [ ]:
sfm_snow_free, sfm_snow_on = sfm_snow_dsm()
sfm_snow_free_values = sfm_snow_free.band_values()
sfm_snow_on_values = sfm_snow_on.band_values()

np.ma.masked_where(control_surfaces_values.mask, sfm_snow_free_values, copy=False)
np.ma.masked_where(control_surfaces_values.mask, sfm_snow_on_values, copy=False);

### Percent Control surface pixels in DEM 

In [ ]:
print(f"Percent of available: {control_surfaces_values.count() / control_surfaces_values.size:.2%}")

In [ ]:
def compare_control_surfaces(data):
    mad = MedianAbsoluteDeviation(
        data[np.isfinite(data)].compressed()
    )
    
    make_box_plot(data, 'Control Surfaces');
    
    plot_opts = [
        {
            'data': data,
            'label': 'Control Surfaces',
            'color': 'peru',
            'legend': LEGEND_TEXT.format(" NMAD", mad.normalized(), 'm'),
            'skip_mean': True,
            'alpha': 0.4
        }
    ]

    with Histogram.plot(plot_opts, (-2, 2), figsize=(8, 3)) as ax:
        ax
        
    plt.figure(figsize=(12, 12))
    plt.imshow(
        data, 
        extent=sfm_snow_free.extent,
        cmap=RED_BLUE_CMAP,
        vmin=-.5,
        vmax=.5,
    )

    ax = plt.gca()
    ax.set_xticks([])
    ax.set_yticks([])
    ax.patch.set_color('grey')

    ax.add_artist(ScaleBar(1.0, location='lower left'))
    plt.colorbar();


### Control Surfaces: Snow On - Snow Free 

In [ ]:
compare_control_surfaces((sfm_snow_on_values - sfm_snow_free_values))

### Control surfaces: Reference DEM - Snow Free

In [ ]:
dem_co_reg_values = load_co_reg_dem()

np.ma.masked_where(control_surfaces_values.mask, dem_co_reg_values, copy=False);

In [ ]:
compare_control_surfaces((dem_co_reg_values - sfm_snow_free_values))

### Control Surfaces: Reference DEM - Snow On

In [ ]:
compare_control_surfaces((dem_co_reg_values - sfm_snow_on_values))